In [1]:
import cv2
import mediapipe as mp
import socket

# MediaPipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

# TCP 서버 설정
def send_hand_landmarks(host='127.0.0.1', port=6521):
    # 소켓 객체 생성
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        # 서버에 연결
        client_socket.connect((host, port))
        print(f"서버에 연결되었습니다. {host}:{port}")

        while True:
            # 웹캠에서 프레임 읽기
            success, image = cap.read()
            if not success:
                continue

            # 이미지를 RGB로 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 성능을 위해 이미지 쓰기 불가능으로 설정
            image.flags.writeable = False

            # MediaPipe를 통해 손 검출
            results = hands.process(image)

            # 이미지를 RGB에서 BGR로 변환하여 그리기 작업 준비
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # 검출된 손의 랜드마크를 이미지에 그림
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))
                    
                    # 엄지 끝점 랜드마크 가져오기
                    thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                    # 검지 끝점 랜드마크 가져오기
                    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    # 손목 랜드마크 가져오기
                    wrist_landmark = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
                    
                    # 랜드마크들의 좌표 값을 소수점 셋째 자리까지 반올림하여 문자열로 변환하여 서버로 전송
                    landmarks_str = f"{round(thumb_tip.x, 3)},{round(thumb_tip.y, 3)},{round(thumb_tip.z, 3)};" \
                                    f"{round(index_tip.x, 3)},{round(index_tip.y, 3)},{round(index_tip.z, 3)};" \
                                    f"{round(wrist_landmark.x, 3)},{round(wrist_landmark.y, 3)},{round(wrist_landmark.z, 3)}"
                    client_socket.sendall(landmarks_str.encode())

            # 결과 이미지 표시
            cv2.imshow('Hand Tracking', image)

            # 'q'를 눌러 종료
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    send_hand_landmarks()

서버에 연결되었습니다. 127.0.0.1:6521
